In [1]:
import torch
torch.set_default_tensor_type(torch.DoubleTensor)

import argparse
from   config            import *
from   UKDALE_Parser     import *
from   REDD_Parser       import *
from   Refit_Parser      import *
from PChainger_Parser import PChainger_Parser
from   Electricity_model import *
from   NILM_Dataloader   import *
from   Trainer           import *
from   time              import time
import pickle            as pkl
from pathlib import Path
import plotly.express as px
import plotly.io as pio

pio.renderers.default = 'iframe' # or 'colab' or 'iframe' or 'iframe_connected' or 'sphinx_gallery'

In [2]:
with open("./results_UK-DALE_TitanV_kettle/uk_dale/kettle/results.pkl", "rb") as f:
    res = pkl.load(f)

args = res["args"]

In [3]:
# override computing device
args.device = "cpu"
args.pretrain_num_epochs = 0
args.num_epochs = 0
args.validation_size = 1
args.hidden = 256

In [4]:
setup_seed(args.seed)

In [5]:
args.house_indices = [2]
args.validation_size = .1

ds_parser = PChainger_Parser(
    sampling=args.sampling,
    normalize="mean",
    cutoff=[3100],
    #threshold=[2000],
    location="./data/test/geert/main.csv",
    #min_on=[2],
    #min_off=[0],
    separator=",",
    val_size=1,
    window_size=480,
    window_stride=240,
    drop_last=True,
)

In [6]:
model = ELECTRICITY(args)
args.pretrain = False  # remove?
model.pretrain = False  # remove?
model.load_state_dict(torch.load("./results_UK-DALE_TitanV_kettle/uk_dale/kettle/best_acc_model.pth", map_location=torch.device('cpu')))
trainer = Trainer(args,ds_parser,model)

AttributeError: 'PChainger_Parser' object has no attribute 'val_size'

In [ ]:
ds_uk_parser = UK_Dale_Parser(args)

#print(len(ds_uk_parser.x))
#print(ds_uk_parser.x[:100])
#print(type(ds_uk_parser.x))
#print(min(ds_uk_parser.x))
#print(max(ds_uk_parser.x))

#print(len(ds_parser.x))
#print(ds_parser.x[:100])
#print(type(ds_parser.x))
#print(min(ds_parser.x))
#print(max(ds_parser.x))

In [ ]:
dataloader = NILMDataloader(args, ds_uk_parser)
_, test_loader = dataloader.get_dataloaders()
mre, mae, acc, prec, recall, f1 = trainer.test(test_loader, map_location='cpu')

In [ ]:
print('Mean Accuracy:', acc)
print('Mean F1-Score:', f1)
print('MAE:', mae)
print('MRE:', mre)

## "Own" data

In [ ]:
inf_data_start = 0
inf_data_end = 480
inference_data = NILMDataset(
    #x=ds_uk_parser.x[-100000:],
    x=ds_parser.x[-100000:],
    #y=ds_uk_parser.y[-100000:],
    y=[0] * 100000,
    #status=ds_uk_parser.status[-100000:],
    status=[0] * 100000,
    window_size=480,
    stride=480
)

In [ ]:
inference_dataloader = torch.utils.data.DataLoader(
    dataset=inference_data,
    batch_size=inf_data_end-inf_data_start,
)

In [ ]:
out = model.eval()

In [ ]:
batches = []
for batch in inference_dataloader:
    batches.append(model(batch[0].to("cpu")))

data = []
for batch in batches:
    for chunk in batch[0]:
        data += list(chunk[0].tolist())

In [ ]:
df_inf = pd.DataFrame(dict(
    #index = range(len(data)),
    #inf = data
    index = range(10000),
    inf = data[-10000:]
))

fig = px.line(df_inf, x="index", y="inf", line_shape="linear", render_mode="svg")
fig.show()

## "Orig" data

In [ ]:
inf_data_start = 0
inf_data_end = 480
inference_data = NILMDataset(
    x=ds_uk_parser.x[-100000:],
    #x=ds_parser.x[-100000:],
    #y=ds_uk_parser.y[-100000:],
    y=[0] * 100000,
    #status=ds_uk_parser.status[-100000:],
    status=[0] * 100000,
    window_size=480,
    stride=480
)

In [ ]:
inference_dataloader = torch.utils.data.DataLoader(
    dataset=inference_data,
    batch_size=inf_data_end-inf_data_start,
)

In [ ]:
out = model.eval()

In [ ]:
batches = []
for batch in inference_dataloader:
    batches.append(model(batch[0].to("cpu")))

data = []
for batch in batches:
    for chunk in batch[0]:
        data += list(chunk[0].tolist())

In [ ]:
df_inf = pd.DataFrame(dict(
    #index = range(len(data)),
    #inf = data
    index = range(10000),
    inf = data[-10000:]
))

fig = px.line(df_inf, x="index", y="inf", line_shape="linear", render_mode="svg")
fig.show()